### 数据集
- [Caltech-UC San Diego 数据集](http://www.vision.caltech.edu/visipedia/CUB-200-2011.html)
- 11788张图片，总共分为200类

### 导入相关库

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from os import listdir
from keras.applications.inception_v3 import InceptionV3
from keras.models import load_model,Model
from keras.layers import GlobalAveragePooling2D,Dense
from keras.optimizers import SGD
from keras.utils import np_utils
from keras.preprocessing import image
from keras.callbacks import TensorBoard

Using TensorFlow backend.


### 预处理

- 转换为256*256像素的图像，统一处理
- 通道数为3

In [2]:
ROWS = 256
COLS = 256
CHANNELS = 3

- 类名

In [3]:
CLASS_NAMES = sorted(listdir('CUB_200_2011/images'))
len(CLASS_NAMES)

200

In [4]:
CLASS_NAMES[:5]

['001.Black_footed_Albatross',
 '002.Laysan_Albatross',
 '003.Sooty_Albatross',
 '004.Groove_billed_Ani',
 '005.Crested_Auklet']

### 导入数据

- 图像比较少，使用ImageDataGenerato产生更多图像：水平翻转、缩放、旋转
- 将原始图像分割为训练集和测试集，都包含200类

In [5]:
train_image_generator = image.ImageDataGenerator(horizontal_flip=True, rescale=1./255, rotation_range=45)
test_image_generator = image.ImageDataGenerator(horizontal_flip=False, rescale=1./255, rotation_range=0)

train_generator = train_image_generator.flow_from_directory('CUB_200_2011/train', target_size=(ROWS, COLS), class_mode='categorical')
test_generator = test_image_generator.flow_from_directory('CUB_200_2011/test', target_size=(ROWS, COLS), class_mode='categorical')

Found 8140 images belonging to 200 classes.
Found 3648 images belonging to 200 classes.


### 搭建模型

- 使用ImageNet的Inceptionv3模型
- 去掉顶层
- 整个卷积网络求平均
- 全连接1024 ==》 200

In [6]:
base_model = InceptionV3(weights='imagenet', include_top=False)

x = base_model.output
x = GlobalAveragePooling2D()(x)

x = Dense(1024, activation='relu')(x)
out_layer = Dense(200, activation='softmax')(x)

model = Model(inputs=base_model.input, outputs=out_layer)

### 训练模型

- compile
- SGD优化函数，学习率0.0001

In [7]:
for layer in model.layers:
    layer.trainable = True

model.compile(optimizer=SGD(lr=0.001, momentum=0.9), loss='categorical_crossentropy', metrics=['accuracy'])

model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, None, None, 3 0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, None, None, 3 864         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, None, None, 3 96          conv2d_1[0][0]                   
__________________________________________________________________________________________________
activation_1 (Activation)       (None, None, None, 3 0           batch_normalization_1[0][0]      
__________________________________________________________________________________________________
conv2d_2 (

- 迭代100次

In [8]:
tensorboard = TensorBoard(log_dir='./logs/inceptionv3')
model.fit_generator(train_generator, steps_per_epoch=32, epochs=100, callbacks=[tensorboard])

Epoch 1/100
32/32 [==============================] - 20s 638ms/step - loss: 5.3640 - acc: 0.0020
Epoch 2/100
32/32 [==============================] - 12s 363ms/step - loss: 5.2916 - acc: 0.0059
Epoch 3/100
32/32 [==============================] - 12s 363ms/step - loss: 5.2494 - acc: 0.0176
Epoch 4/100
32/32 [==============================] - 12s 364ms/step - loss: 5.1479 - acc: 0.0342
Epoch 5/100
32/32 [==============================] - 12s 363ms/step - loss: 5.0745 - acc: 0.0518
Epoch 6/100
32/32 [==============================] - 12s 364ms/step - loss: 4.9486 - acc: 0.0752
Epoch 7/100
32/32 [==============================] - 12s 364ms/step - loss: 4.7821 - acc: 0.1016
Epoch 8/100
32/32 [==============================] - 12s 367ms/step - loss: 4.6259 - acc: 0.1191
Epoch 9/100
32/32 [==============================] - 12s 373ms/step - loss: 4.3449 - acc: 0.1787
Epoch 10/100
32/32 [==============================] - 12s 376ms/step - loss: 4.1247 - acc: 0.1982
Epoch 11/100
32/32 [=========

32/32 [==============================] - 12s 374ms/step - loss: 0.4321 - acc: 0.8848
Epoch 85/100
32/32 [==============================] - 12s 380ms/step - loss: 0.4274 - acc: 0.8926
Epoch 86/100
32/32 [==============================] - 12s 376ms/step - loss: 0.4196 - acc: 0.8936
Epoch 87/100
32/32 [==============================] - 12s 378ms/step - loss: 0.3970 - acc: 0.9036
Epoch 88/100
32/32 [==============================] - 12s 381ms/step - loss: 0.3920 - acc: 0.9023
Epoch 89/100
32/32 [==============================] - 12s 381ms/step - loss: 0.3312 - acc: 0.9268
Epoch 90/100
32/32 [==============================] - 12s 373ms/step - loss: 0.3775 - acc: 0.9027
Epoch 91/100
32/32 [==============================] - 12s 379ms/step - loss: 0.3391 - acc: 0.9209
Epoch 92/100
32/32 [==============================] - 12s 383ms/step - loss: 0.3674 - acc: 0.9141
Epoch 93/100
32/32 [==============================] - 12s 380ms/step - loss: 0.3709 - acc: 0.9111
Epoch 94/100
32/32 [=============

### 评估模型

In [9]:
print(model.evaluate_generator(test_generator, steps=5000))

[0.8777411760210991, 0.75843125]


### 保存模型

In [10]:
model.save("birds-inceptionv3.model")

### 预测

In [11]:
model = load_model('birds-inceptionv3.model')

def predict(fname):
    img = image.load_img(fname, target_size=(ROWS, COLS))
    img_tensor = image.img_to_array(img) # (height, width, channels)
    # (1, height, width, channels), add a dimension because the model expects this shape:
    # (batch_size, height, width, channels)
    img_tensor = np.expand_dims(img_tensor, axis=0) 
    img_tensor /= 255. # model expects values in the range [0, 1]
    prediction = model.predict(img_tensor)[0]
    best_score_index = np.argmax(prediction)
    bird = CLASS_NAMES[best_score_index] # retrieve original class name
    print("Prediction: %s (%.2f%%)" % (bird, 100*prediction[best_score_index]))

In [14]:
predict('Green_Kingfisher_0028_70981.jpg')
predict('Rhinoceros_Auklet_0004_797541.jpg')
predict('Vermilion_Flycatcher_0002_42390.jpg')

Prediction: 080.Green_Kingfisher (33.13%)
Prediction: 008.Rhinoceros_Auklet (75.45%)
Prediction: 042.Vermilion_Flycatcher (97.40%)
